In [1]:
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import Dataset
#from torchvision import Datasets
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import random
import spacy
import torch.nn.functional as F
import os

C:\Users\maddo\anaconda3\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
spacy_en = spacy.load('en_core_web_sm')
spacy_fr = spacy.load('fr_core_news_sm')

In [3]:
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

def tokenize_fr(text):
    return [tok.text for tok in spacy_fr.tokenizer(text)]

In [4]:
data=pd.read_csv('engfrench.csv')

In [5]:
print(len(data))

175621


In [6]:
data.iloc[170000]

English words/sentences    The guards found a hacksaw blade in the prison...
French words/sentences     Les gardiens trouvèrent une lame de scie à mét...
Name: 170000, dtype: object

In [7]:
data[10000:10005]

,English words/sentences,French words/sentences
10000,Be very careful.,Sois très prudente !
10001,Bees make honey.,Les abeilles font du miel.
10002,Behave yourself.,Comporte-toi bien.
10003,Bite the bullet.,Serre les dents.
10004,Bite the bullet.,Serrez les dents.


from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")

In [8]:
PAD_TOKEN_ID=0
SOS_TOKEN_ID = 1
EOS_TOKEN_ID = 2

In [9]:
txten=data.iloc[0,0]
txtfr=data.iloc[0,1]


In [10]:
txten

'Hi.'

In [11]:
txtfr

'Salut!'

In [12]:
en_check=torch.load('engEmbeddings.pt')
fr_check=torch.load('frEmbeddings.pt')
en_weights=en_check['weights']
en_vocab=en_check['vocab_dict']
fr_weights=fr_check['weights']
fr_vocab=fr_check['vocab_dict']

C:\Users\maddo\AppData\Local\Temp\ipykernel_19472\1505539583.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  en_check=torch.load('engEmbeddings.pt')
C:\Users\maddo\AppDa

In [13]:
class TranslationDataset(Dataset):
    def __init__(self, dataframe, en_vocab, fr_vocab):
        self.df = dataframe
        self.en_vocab = en_vocab
        self.fr_vocab = fr_vocab
        
        # Define special token IDs
        # IMPORTANT: Ensure these keys exist in your vocab dictionaries!
        self.unk_idx_en = en_vocab.get('<unk>', 0) # Default to 0 if not found
        self.eos_idx_en = en_vocab.get('<eos>', 2)
        
        self.unk_idx_fr = fr_vocab.get('<unk>', 0)
        self.sos_idx_fr = fr_vocab.get('<sos>', 1)
        self.eos_idx_fr = fr_vocab.get('<eos>', 2)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # 1. Get Raw Text
        src_text = str(self.df.iloc[index, 0])
        trg_text = str(self.df.iloc[index, 1])

        # 2. Tokenize (using your Spacy functions)
        # We use .lower() because FastText matches better with lowercase
        src_tokens = tokenize_en(src_text.lower())
        trg_tokens = tokenize_fr(trg_text.lower())

        # 3. Numericalize Source (English)
        # Use .get() to map unknown words to <unk> index
        src_indices = [self.en_vocab.get(token, self.unk_idx_en) for token in src_tokens]
        
        # 4. Numericalize Target (French)
        trg_indices = [self.fr_vocab.get(token, self.unk_idx_fr) for token in trg_tokens]

        # 5. Add Special Tokens
        # Source usually needs <eos> at the end so the LSTM knows the sentence stopped
        src_indices.append(self.eos_idx_en)
        
        # Target needs <sos> to start generation and <eos> to end it
        trg_indices = [self.sos_idx_fr] + trg_indices + [self.eos_idx_fr]

        return torch.tensor(src_indices, dtype=torch.long), torch.tensor(trg_indices, dtype=torch.long)

In [14]:
datanewdataset=TranslationDataset(data,en_vocab,fr_vocab)

In [15]:
def custom_collate_fn(batch):
    # 'batch' is a list of tuples: [(input_tensor_1, target_tensor_1), (input_tensor_2, target_tensor_2), ...]

    # 1. Separate inputs and targets
    inputs = [torch.tensor(item[0]) for item in batch]
    targets = [torch.tensor(item[1]) for item in batch]

    # 2. Pad the inputs and targets to the length of the longest sequence in the batch
    # batch_first=True makes the output shape (BatchSize, SequenceLength)
    inputs_padded = pad_sequence(inputs, batch_first=True, padding_value=PAD_TOKEN_ID)
    targets_padded = pad_sequence(targets, batch_first=True, padding_value=PAD_TOKEN_ID)
    
    return inputs_padded, targets_padded

In [16]:
haha=DataLoader(datanewdataset,batch_size=32,shuffle=True,collate_fn=custom_collate_fn)

SAMPLE TO REMEMBER LSTM 

In [17]:
xa=torch.randn(10,100)

In [18]:
ha=nn.LSTM(100,200)

In [19]:
ya=ha(xa)

In [20]:
ya[0].shape

torch.Size([10, 200])

In [21]:
ya[1][0].shape

torch.Size([1, 200])

In [22]:
ya[1][1].shape

torch.Size([1, 200])

In [23]:
embed_dim=300
#since using pre trained Fast TExt aligned vectors

In [24]:
VOCAB_SIZE_ENC=len(en_vocab)

In [25]:
VOCAB_SIZE_FR=len(fr_vocab)

In [26]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Create a long constant matrix representing position
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # Register as buffer (not a learnable parameter, but part of state_dict)
        self.register_buffer('pe', pe.unsqueeze(0)) 

    def forward(self, x):
        # x shape: [batch_size, seq_len, d_model]
        # Add position encoding to embeddings
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

In [27]:
class TransformerSeq2Seq(nn.Module):
    def __init__(self, device, embed_dim=300, nhead=4, num_layers=3, dropout=0.1):
        super().__init__()
        self.device = device
        
        # 1. Embeddings (Using your pretrained weights)
        self.src_emb = nn.Embedding.from_pretrained(en_weights, freeze=True)
        self.tgt_emb = nn.Embedding.from_pretrained(fr_weights, freeze=True)
        
        # 2. Positional Encoding
        self.positional_encoding = PositionalEncoding(embed_dim, dropout=dropout)
        
        # 3. The Transformer
        # batch_first=True is crucial to match your [batch, len] data format
        self.transformer = nn.Transformer(
            d_model=embed_dim,
            nhead=nhead,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=2048,
            dropout=dropout,
            batch_first=True 
        )
        
        # 4. Output Layer
        self.fc_out = nn.Linear(embed_dim, len(fr_vocab)) # Project to French Vocab size

    def forward(self, src, tgt):
        # src: [batch, src_len]
        # tgt: [batch, tgt_len] (This is the "Teacher Forcing" input)
        
        # Create Masks (Crucial for Transformers!)
        # src_mask = mask for padding tokens in source
        # tgt_mask = "Causal Mask" (prevents decoder from seeing future words) + padding mask
        tgt_seq_len = tgt.shape[1]
        tgt_mask = self.transformer.generate_square_subsequent_mask(tgt_seq_len).to(self.device)
        
        # Generate Padding Masks (assuming 0 is your pad_token_id)
        # These boolean masks tell the model to ignore padding tokens
        src_padding_mask = (src == 0).to(self.device) 
        tgt_padding_mask = (tgt == 0).to(self.device)

        # Apply Embeddings + Positional Encoding
        src_emb = self.positional_encoding(self.src_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_emb(tgt))
        
        # Pass through Transformer
        outs = self.transformer(
            src=src_emb,
            tgt=tgt_emb,
            tgt_mask=tgt_mask, # Mask future positions
            src_key_padding_mask=src_padding_mask,
            tgt_key_padding_mask=tgt_padding_mask,
            memory_key_padding_mask=src_padding_mask # Mask source padding in attention
        )
        
        # Final Projection
        return self.fc_out(outs)

In [35]:
def train_one_epoch(model, iterator, optimizer, criterion, epoch):
    model.train()
    epoch_loss = 0
    
    # RTX 4070 supports Mixed Precision (faster training)
    scaler = torch.cuda.amp.GradScaler() 

    for i, (src, trg) in enumerate(iterator):
        src = src.to(device)
        trg = trg.to(device) # trg shape: [batch, trg_len] e.g., <sos> ... <eos>

        optimizer.zero_grad()
        
        # Prepare Inputs and Targets for Transformer
        # Decoder Input: Remove the LAST token (<eos>)
        tgt_input = trg[:, :-1] 
        
        # Expected Output: Remove the FIRST token (<sos>)
        tgt_output = trg[:, 1:] 
        
        with torch.cuda.amp.autocast(): # Mixed precision context
            # Forward pass
            output = model(src, tgt_input)
            
            # Output shape: [batch, seq_len-1, vocab_size]
            # Reshape for Loss: [batch * (seq_len-1), vocab_size]
            output = output.reshape(-1, output.shape[-1])
            tgt_output = tgt_output.reshape(-1)
            
            loss = criterion(output, tgt_output)

        # Backprop with Scaler
        scaler.scale(loss).backward()
        
        # Clip Gradients (Good stability for Transformers too)
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        
        scaler.step(optimizer)
        scaler.update()

        epoch_loss += loss.item()

        if i % 500 == 0:
            print(f"Epoch {epoch} Batch {i}, Loss: {loss.item():.4f}")

    print(f"Mean Epoch Loss: {epoch_loss / len(iterator)}")

In [36]:
def predict_translation_transformer(model, sentence, max_len=50):
    model.eval()
    
    # 1. Preprocess (Tokenize + Numericalize)
    tokens = tokenize_en(sentence.lower())
    unk_idx = en_vocab.get('<unk>', 3)
    eos_idx = en_vocab.get('<eos>', 2)
    ids = [en_vocab.get(token, unk_idx) for token in tokens]
    ids.append(eos_idx) # Add <eos> to source
    
    src_tensor = torch.tensor(ids, dtype=torch.long).unsqueeze(0).to(device)
    
    # 2. The Loop
    # Start with just the <sos> token
    sos_idx = fr_vocab['<sos>']
    eos_idx = fr_vocab['<eos>']
    
    tgt_indices = [sos_idx]
    
    for _ in range(max_len):
        tgt_tensor = torch.tensor(tgt_indices, dtype=torch.long).unsqueeze(0).to(device)
        
        with torch.no_grad():
            # Get masks inside the model forward (or create dummy masks here)
            # We call the model. The model will create the causal mask internally
            output = model(src_tensor, tgt_tensor)
        
        # We only care about the prediction for the LAST token
        # output shape: [1, len, vocab]
        last_token_logits = output[:, -1, :]
        pred_token = last_token_logits.argmax(1).item()
        
        tgt_indices.append(pred_token)
        
        if pred_token == eos_idx:
            break
            
    # 3. Decode to Words
    idx_to_word = {v: k for k, v in fr_vocab.items()}
    predicted_tokens = [idx_to_word.get(idx, '<unk>') for idx in tgt_indices[1:]] # Skip <sos>
    
    # Remove <eos> from display if present
    if predicted_tokens and predicted_tokens[-1] == '<eos>':
        predicted_tokens = predicted_tokens[:-1]
        
    return " ".join(predicted_tokens)

In [37]:
import math

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Running on: {torch.cuda.get_device_name(0)}") # Should say RTX 4070

model = TransformerSeq2Seq(
    device=device,
    embed_dim=embed_dim, # e.g. 300
    nhead=4,             # 300 / 4 = 75
    num_layers=3,
    dropout=0.1
).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss(ignore_index=0) # Assuming 0 is pad_index

# Run Loop


Running on: NVIDIA GeForce RTX 4070


In [39]:
for epoch in range(20):
    train_one_epoch(model,haha, optimizer, criterion, epoch)

C:\Users\maddo\AppData\Local\Temp\ipykernel_19472\3600398038.py:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
C:\Users\maddo\AppData\Local\Temp\ipykernel_19472\1408414250.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = [torch.tensor(item[0]) for item in batch]
C:\Users\maddo\AppData\Local\Temp\ipykernel_19472\1408414250.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = [torch.tensor(item[1]) for item in batch]
C:\Users\maddo\AppData\Local\Temp\ipykernel_19472\3600398038.py:21: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. P

Epoch 0 Batch 0, Loss: 10.2131
Epoch 0 Batch 500, Loss: 5.1015
Epoch 0 Batch 1000, Loss: 4.5509
Epoch 0 Batch 1500, Loss: 4.4730
Epoch 0 Batch 2000, Loss: 4.4362
Epoch 0 Batch 2500, Loss: 4.2060
Epoch 0 Batch 3000, Loss: 3.8827
Epoch 0 Batch 3500, Loss: 4.1893
Epoch 0 Batch 4000, Loss: 4.0161
Epoch 0 Batch 4500, Loss: 3.6453
Epoch 0 Batch 5000, Loss: 3.6865
Mean Epoch Loss: 4.175788068753988
Epoch 1 Batch 0, Loss: 3.2777
Epoch 1 Batch 500, Loss: 3.3258
Epoch 1 Batch 1000, Loss: 3.3926
Epoch 1 Batch 1500, Loss: 3.2175
Epoch 1 Batch 2000, Loss: 3.0209
Epoch 1 Batch 2500, Loss: 3.1661
Epoch 1 Batch 3000, Loss: 3.4206
Epoch 1 Batch 3500, Loss: 2.8804
Epoch 1 Batch 4000, Loss: 2.5529
Epoch 1 Batch 4500, Loss: 2.7795
Epoch 1 Batch 5000, Loss: 2.7276
Mean Epoch Loss: 3.0520717572983465
Epoch 2 Batch 0, Loss: 2.0434
Epoch 2 Batch 500, Loss: 2.9962
Epoch 2 Batch 1000, Loss: 2.7126
Epoch 2 Batch 1500, Loss: 2.6753
Epoch 2 Batch 2000, Loss: 2.2042
Epoch 2 Batch 2500, Loss: 2.4954
Epoch 2 Batch 30

In [60]:
torch.save({
    'epoch': 11,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, "TransformerFR_ENGMODELANDALL.pth")

In [ ]:
torch.save({'just_model':model.state_dict()},"modelkek.pth")

In [ ]:
predict_translation_transformer

In [43]:
translation = predict_translation_transformer(model, "I am doing great , WHat about you?")
print(f"Translation: {translation}")

Translation: je fais bien , de quoi vous ?


In [44]:
translation = predict_translation_transformer(model, "Today's a good day.")
print(f"Translation: {translation}")

Translation: aujourd'hui est un bon jour .


In [45]:
translation = predict_translation_transformer(model, "How was prison?")
print(f"Translation: {translation}")

Translation: comment s' est -elle échappé ?


In [46]:
translation = predict_translation_transformer(model, "Bite the bullet.")
print(f"Translation: {translation}")

Translation: mords la flèche .


In [47]:
translation = predict_translation_transformer(model, "what's the time?")
print(f"Translation: {translation}")

Translation: quelle est le temps ?


In [48]:
translation = predict_translation_transformer(model, "what's the time")
print(f"Translation: {translation}")

Translation: quelle est le temps .


In [49]:
translation = predict_translation_transformer(model, "whats the time?")
print(f"Translation: {translation}")

Translation: à quoi remonte le temps ?


In [50]:
translation = predict_translation_transformer(model, "behave yourself!")
print(f"Translation: {translation}")


Translation: fais attention !


In [51]:
translation = predict_translation_transformer(model, "please stop!")
print(f"Translation: {translation}")

Translation: arrête - toi , je te prie !


In [52]:
translation = predict_translation_transformer(model, "please stop")
print(f"Translation: {translation}")

Translation: arrête , s' il te plaît .


In [53]:
translation = predict_translation_transformer(model, "Be very careful of bees.")
print(f"Translation: {translation}")

Translation: soyez très prudentes des abeilles .


In [54]:
translation = predict_translation_transformer(model, "It's all nonsense.")
print(f"Translation: {translation}")

Translation: c' est tout faux .


In [55]:

data[22000:22005]

,English words/sentences,French words/sentences
22000,Why is this funny?,Pourquoi c'est marrant ?
22001,Why must I suffer?,Pourquoi dois-je souffrir ?
22002,Why must we do it?,Pourquoi devons-nous le faire ?
22003,Why not just quit?,Pourquoi ne pas simplement démissionner ?
22004,Why not just quit?,Pourquoi ne pas simplement arrêter ?


In [56]:
translation = predict_translation_transformer(model, "It's a funny country.")
print(f"Translation: {translation}")

Translation: c' est un pays marrant .


C:\Users\maddo\anaconda3\Lib\site-packages\torch\nn\functional.py:5849: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


In [57]:
translation = predict_translation_transformer(model, "It's a distraction.")
print(f"Translation: {translation}")

Translation: c' est une distraction .


In [58]:
translation = predict_translation_transformer(model, "Why not just quit?")
print(f"Translation: {translation}")

Translation: pourquoi ne pas simplement arrêter ?


In [59]:
translation = predict_translation_transformer(model, "Why must I suffer?")
print(f"Translation: {translation}")


Translation: pourquoi dois -je souffrir ?
